In [1]:
import pandas as pd
from datetime import date
from time import sleep

import zipfile
import os

In [2]:
## diretorio de trabalho e armazenamento de dados
wd = r"/home/yair/GHub/Codigos-em-financas/data/"

## remove planilha caso exista
try:
    os.remove(wd + "setores.csv")
except: pass

In [3]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
chrome_opts = Options()
chrome_opts.add_experimental_option("prefs", {
  "download.default_directory": wd,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True
})

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [4]:
driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()), #
                          options=chrome_opts)
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'
driver.get(url)

#### navegação

In [5]:
popup = '//*[@id="onetrust-close-btn-container"]/button' ## '/html/body/div[2]/div[3]/div/div[2]/button'
driver.find_element('xpath', popup).click()

WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "bvmf_iframe")))

botao_expandir = '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[1]/div/div/a/i'
driver.execute_script("arguments[0].click();", driver.find_element('xpath', botao_expandir))

planilha = driver.find_element('xpath', '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[2]/div/app-companies-home-filter-classification/form/div[2]/div[3]/div[2]/p/a')
driver.execute_script("arguments[0].click();", planilha)
sleep(5)

driver.quit()

#### extrair o ZIP

In [6]:
arquivo_zip = zipfile.ZipFile( wd + r"ClassifSetorial.zip")

for planilha in arquivo_zip.namelist():
    setores = pd.read_excel(arquivo_zip.open(planilha), skiprows=6)

arquivo_zip.close()
os.remove(wd + r"ClassifSetorial.zip")


#### prepara planilha

In [7]:
setores['SUBSETOR'] = setores['SUBSETOR'].ffill()
setores = setores.dropna(subset = ['SEGMENTO'])
setores = setores[['SUBSETOR', 'SEGMENTO', 'LISTAGEM']]

setores = setores.dropna()
setores.columns = ['SETOR', 'NOME', 'TICKER']

setores.to_csv(wd+"setores.csv", index = False)

